In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

print('imports ok')

## Exploring dataset

In [ ]:
df = pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.describe(include="all")

## import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import HTML
import calendar
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS

In [ ]:
# Missing Values

missing_value = 100 * df.isnull().sum()/len(df)
missing_value = missing_value.reset_index()
missing_value.columns = ['variables', 'missing values in percentage']

# heatmap
fig = px.imshow(df.isnull().T, template='ggplot2')
fig.update_layout(title='Missing values in dara set')
fig.show()

In [ ]:
# barplot
fig = px.bar(missing_value, y='missing values in percentage', x='variables', title='Missing values % in each column', template='ggplot2')
fig.show()


* Mutias das variáveis possuem NaN/Valores nulos como People_cavvinations_raw e People_fully_vaccinations possuem mais de 80% de valores nulos
* Dayly_vaccinations, total_vaccinations possuem valores nulos porque se tomarmos em consideração que alguns países não iniciaram a vacinação a partir da data em 2020

# Limpeza dos dados

In [ ]:
# passando data para formato pt-BR

# df['date'] = pd.to_datetime(df['date'], format('%d/%m/%y'))
# df['date']

# df['date'] = df['date'] = pd.to_datetime(df.date, infer_datetime_format=True)
# df.head(100)
# df['date'] = df['date'].dt.strftime(format = '%d/%m/%y')

In [ ]:
df.head()

In [ ]:
# Limpeza de ISO code dados
df[df['iso_code'].isnull()]['country'].unique()

# países que estão bob código do Reino Unido

In [ ]:
df['iso_code'] = df['iso_code'].fillna('GBR')

In [ ]:

profile = ProfileReport(df, title="Pandas Profiling Report")

profile

# Quais países usam qual vacina?

In [ ]:
df_vaccines = df.groupby(['country', 'vaccines'])[['people_vaccinated']].max().reset_index()
df_vaccines

In [ ]:
df_vaccines = df_vaccines.sort_values('people_vaccinated', ascending=False)
df_vaccines.columns = ['Country','Vaccines','Total number of people vaccinated']
df_vaccines.head(10)

In [ ]:
fig = px.sunburst(
    df_vaccines, path=['Vaccines', 'Country'],
    values='Total number of people vaccinated',
    title='Quais países usam quais vacinas?',
    height=650, color_continuous_scale=px.colors.cyclical.Twilight
)

fig.show()

* no plot acima, podemos perceber no centro, os tipos de vacinas e ao clicar nelas, podemos ver quais países mais a usam. 

In [ ]:
plot_vaccines = df.groupby(['vaccines'])[['people_vaccinated']].max().reset_index()
plot_vaccines

In [ ]:
plot_vaccines = plot_vaccines.sort_values('people_vaccinated', ascending=False)
plot_vaccines

In [ ]:
plot_vaccines.columns = ['Vaccines', 'Total number of people vaccinated']

fig = px.bar(plot_vaccines, x = 'Vaccines', y='Total number of people vaccinated', template='none', title='Top Vaccines', color='Total number of people vaccinated')
fig.update(layout_coloraxis_showscale=False)
fig.show()

# Em qual país o programa de vacinação está mais avançado?

In [ ]:
# Top countries in Vaccination

fig = px.bar(df_vaccines[0:24], x='Country', y='Total number of people vaccinated', template='none', title='Top countries in Vaccination progress', color='Total number of people vaccinated')
fig.update(layout_coloraxis_showscale=True)
fig.show()

In [ ]:
plot_vac = df.groupby('iso_code')['total_vaccinations'].max().reset_index()
plot_vac

In [ ]:
fig = px.choropleth(plot_vac, locations="iso_code",
                   color="total_vaccinations",
                   hover_name='iso_code',
                   color_continuous_scale=px.colors.sequential.Magma,
                   title='Total vaccinations in each country')

fig.update_layout(geo=dict(bgcolor = 'white'))

fig.update_geos(projection_type="natural earth")

fig.show()

* The USa, China and the U.K are the top of the vaccination progress

In [ ]:
date_plot = df.sort_values('date', ascending=True)


In [ ]:
fig = px.choropleth(date_plot, locations='iso_code',
                   color='daily_vaccinations',
                   animation_frame = 'date',
                   hover_name='country',
                   title='Daily vaccinations in each Country')

fig.update_layout(geo=dict(bgcolor = 'White'))
fig.update_geos(projection_type='natural earth')
fig.show()

* Alguns países iniciaram o programa de vacinação em Dezembro de 2020. (Rússia, EUA, China, Argentina, México e UK)

* Alguns países iniciaram o programa de vacinação em meados de Janeiro. (Indonésia,India, Emirados Árabes Unidos, Brasil, França e Itália)

In [ ]:
percent_vacc = df.groupby(['country', 'iso_code']).max().reset_index()
percent_vacc

In [ ]:
fig = px.scatter_geo(percent_vacc, locations='iso_code',
                    size='total_vaccinations_per_hundred',
                    color='country',
                    hover_name='country',
                    color_continuous_scale=px.colors.sequential.Tealgrn,
                    title="Percentage of people vaccinated per hundred in each Country")

fig.update_layout(geo=dict(bgcolor='#61efff'))
fig.update_geos(projection_type='natural earth')

fig.show()